Zincir bir LLM'in temel yapıtaşıdır . Zincirler bir LLM'ler ile promptların birleştirilmiş halidirler. Sonra bu yapıtaşı bloklar bir araya getirilerek daha büyük bir seri işlemler gerçekleştirilebilir.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # lokal .env dosyasını oku

#!pip install pandas

import pandas as pd
df = pd.read_csv('Data.csv') # Product Review şeklinde 2 sütunu olan bir data: https://s172-31-14-227p20192.lab-aws-production.deeplearning.ai/edit/Data.csv#

df.head()

# LLMChain
LLMChain, LLM ve promptu birleştirip zinciri ortaya çıkaran sınıftır.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature=0.9) # 1'e yakınken daha yaratıcı ve hata ihtimali yüksek sonuçlar verir

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "{product} üreten bir şirketi en iyi tanımlayan isim nedir?"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt) # chain = llm + prompt

In [ ]:
product = "jumbo Çarşaf Takımı"
chain.run(product) # Prompt'u LLM'e iletir

# SimpleSequentialChain (Basit Sıralı Zincir)
SimpleSequentialChain'de bir zincirin çıktısının bir sonraki zincirin girdisi olduğu birden fazla zincir birleştirilir. İki tür sıralı zincir vardır: Tek girdi/çıktısı olan SimpleSequentialChain ve çoklu girdi/çıktısı olan SequentialChain .

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt şablonu 1
first_prompt = ChatPromptTemplate.from_template(
    "{product} üreten bir şirketi en iyi tanımlayan isim nedir?"
)

# 1. Zincir
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt şablonu 2
second_prompt = ChatPromptTemplate.from_template(
    "Şu şirket için 20 kelimelik bir açıklama yazın:{company_name}"
)
# 2. Zincir
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run(product)

# SequentialChain (Sıralı Zincir)

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# 1. Prompt Şablonu: ingilizceye çevir
first_prompt = ChatPromptTemplate.from_template(
    "Aşağıdaki yorumu Türkçeye çevir:"
     "\n\n{Review}"
)
# 1. Zincir: girdi= İnceleme ve çıktı= Turkish_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Turkish_Review"
                    )

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Aşağıdaki incelemeyi 1 cümle ile özetler misiniz:"
     "\n\n{English_Review}"
)
# 2. zincir: girdi= English_Review ve çıktı= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [ ]:
# 3. prompt şablonu: Türkçeye çevir
third_prompt = ChatPromptTemplate.from_template(
    "Aşağıdaki inceleme hangi dildedir:\n\n{Review}"
)
# 3. zincir: girdi= İnceleme ve çıktı= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [ ]:
# 3. prompt şablonu: followup_message
fourth_prompt = ChatPromptTemplate.from_template(
    "Aşağıdaki özete belirtilen dilde devam eden "
    "bir yanıt yazın:"
    "\in\Özet: {summary}\in\Dil: {language}"
)
# 4. zincir: girdi= özet, dil ve çıktı= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [ ]:
# overall_chain: girdi= Review
# ve çıktı= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain(review)

# Router Chain (Yönlendirici Zinciri)

In [ ]:
physics_template = """Sen çok zeki bir fizik profesörüsün. \
Fizikle ilgili soruları özlü ve anlaşılması kolay bir şekilde yanıtlamakta harikasın. \
Bir sorunun cevabını bilmiyorsan, bilmediğini kabul edersin. \

İşte sana bir soru:
{input}"""

math_template = """Sen çok iyi bir matematikçisin. \
Matematik sorularını cevaplamakta harikasın. \
Siz çok iyisiniz çünkü zor problemleri bileşen parçalarına ayırabilir, \
bileşen parçalarına cevap verebilir ve daha sonra daha geniş soruyu yanıtlamak \
için bunları bir araya getirebilirsiniz.

İşte sana bir soru:
{input}"""

history_template = """Sen çok iyi bir tarihçisin. \
Çeşitli tarihsel dönemlerden insanlar, olaylar ve bağlamlar hakkında \
mükemmel bir bilgiye ve anlayışa sahipsiniz. \
Geçmişi düşünme, yansıtma, tartışma, tartışma ve değerlendirme yeteneğine sahipsiniz. \
Tarihsel kanıtlara ve açıklamalarınızı ve yargılarınızı desteklemek için \
bunları kullanma yeteneğine saygı duyuyorsunuz.

İşte sana bir soru:
{input}"""

computerscience_template = """ Başarılı bir bilgisayar bilimcisisiniz. \
Yaratıcılığa, işbirliğine, ileri görüşlülüğe, kendine güvene, \
güçlü problem çözme yeteneklerine, teorileri ve algoritmaları anlamaya ve\
 mükemmel iletişim becerilerine tutkunuz var. \
Kodlama sorularını yanıtlamakta harikasınız. \
Çok iyisin çünkü bir makinenin kolayca yorumlayabileceği zorunlu adımlarla \
bir sorunu nasıl çözeceğini biliyorsun ve zaman karmaşıklığı ile \
mekan karmaşıklığı arasında iyi bir dengeye sahip bir çözümü nasıl \
seçeceğini biliyorsun.

İşte sana bir soru:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "fizik",
        "description": "Fizik sorularını yanıtlamak için iyi",
        "prompt_template": physics_template
    },
    {
        "name": "matematik",
        "description": "Matematik sorularını yanıtlamak için iyi",
        "prompt_template": math_template
    },
    {
        "name": "tarih",
        "description": "Tarih sorularını yanıtlamak için iyi",
        "prompt_template": history_template
    },
    {
        "name": "bilgisayar bilimi",
        "description": "Bilgisayar bilimi sorularını yanıtlamak için iyi",
        "prompt_template": computerscience_template
    }
]

**MultiPromptChain (çoklu prompt zinciri**) birden fazla prompt şablonu arasında yönlendirme yaparken kullanılır.

**LLMRouterChain (LLM yönelndirici zinciri)**, farklı alt zincirler arasında yönlendirme yapmak için bir dil modeli kullanır. Yukarıda verilen açıklama ve adın kullanıldığı yer burasıdır.

**RouterOutputParser (yönlendirici çıktı ayrıştırıcı)**, LLM çıktısını hangi zincirin kullanılması gerektiğini ve bu zincire hangi girdinin kullanılması gerektiğini belirlemek için kullanılabilecek bir sözlüğe ayrıştırır.

**destination_chains**: Yönlendirici zinciri tarafından çağırılan zincirlerdir. Her destination zinciri bir LLM zinciridir.

**default_chain**: Bu zincir, yönlendiricinin hangi zinciri kullanacağına karar veremediğinde kullanılır.

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Bir dil modeline ham metin girişi \
verildiğinde, girdi için en uygun model bilgi istemini seçin. \
Size mevcut istemlerin adları ve istemin en uygun olduğu şeyin açıklaması \
verilecektir. Ayrıca, gözden geçirmenin nihayetinde dil modelinden daha \
iyi bir yanıta yol açacağını düşünüyorsanız, orijinal girdiyi de \
gözden geçirebilirsiniz.

<< FORMATTING >>
Aşağıdaki gibi biçimlendirilmiş bir JSON nesnesine sahip bir işaretleme kod pasajı döndürün:
```json
{{{{
    "destination": string \ kullanılacak bilgi isteminin adı veya "DEFAULT"
    "next_inputs": string \ orijinal girdinin potansiyel olarak değiştirilmiş bir versiyonu
}}}}
```

REMEMBER: "destination", aşağıda belirtilen aday prompt isimlerinden biri OLMALIDIR VEYA giriş, aday promptlardan herhangi biri için uygun değilse "DEFAULT" olabilir. \
REMEMBER: "next_inputs", herhangi bir değişikliğin gerektiğini düşünmüyorsanız yalnızca orijinal girdi olabilir .

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (```json eklemeyi unutma)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("Kara cisim ışıması nedir?") # Otomatik olarak fizik zincirine yönlendirilir
chain.run("2 + 2 nedir?") # Otomatik olarak matematik zincirine yönlendirilir
chain.run("Neden vücudumuzdaki her hücre DNA içerir?") # Otomatik olarak None seçilir ve dil modeline genel bir istek atılır